<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 3, version 0.2,  September 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![titre](images/header.png)

# III A - SBAE - sous-échantillonnage spatialement équilibré
### Extraire un sous-ensemble d'échantillons de clusters K-Means
-------

Ce bloc-notes vous guide tout au long du processus de création d'un sous-échantillon de la série chronologique et des données de modification récupérées dans II. L'objectif est d'obtenir un sous-échantillon statistiquement équilibré qui peut être utilisé pour la collecte de données de formation, et comprend idéalement un pourcentage plus élevé de classes rares telles que la déforestation, la dégradation et le gain par rapport à une approche de sous-échantillonnage aléatoire pur.

### Charger les librairies

In [1]:
# data management
import numpy as np
import pandas as pd
import geopandas as gpd

# clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# sbae internal functionality  
import helpers as h

/usr/local/lib/python3.8/dist-packages/bfast/monitor/opencl/bfastfinal.py:50290: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  'numpy_type': np.bool }


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

### 1 Charger le fichier de résultats du géopackage

La première étape consiste à charger le fichier de résultats du Notebook II de la série de notebooks SBAE. Ce fichier doit contenir les sorties de divers algorithmes de séries chronologiques et peut en outre contenir des extraits de

In [ ]:
df = gpd.read_file('erp_5km/results_Landsat_ndfi_1985-01-01_2000-01-01_2017-01-01.gpkg')

print('Available Columns')
df.columns

In [ ]:
# glob all files in the data augmentation output folder
files = Path('/home/sepal-user/module_results/esbae/Cote_Ivoire_MRV/04_Data_Augmentation/Landsat/').glob('*geojson')

# prepare for parallel execution
files = [[str(file), False] for file in files]

# read files in parallel nad put the in a list
result = py_helpers.run_in_parallel(
    py_helpers.geojson_to_gdf,
    files,
    workers=4,
    parallelization='processes'
)

# concatenate dataframes from result's list
df = pd.concat(result)

### 2 Sélectionnez les colonnes pertinentes pour créer les clusters

Toutes les colonnes des données chargées ne doivent pas entrer dans le processus de clustering, par ex. le point_id ne nous dit rien sur la distribution statistique par rapport au changement. Dans la cellule ci-dessous se trouve une présélection de colonnes qui contiennent potentiellement des informations sur le changement et qui seront donc utiles pour créer des grappes significatives pour un sous-échantillonnage ultérieur.

In [ ]:
# select columsn thata re used by Kmeans
cols_to_cluster = [
    'mon_images',
    #'elevation',
    #'dw_class_mode', 'dw_tree_prob__max',
    #'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean',
    'bfast_magnitude', 'bfast_means', 
    #'lang_tree_height', 
    #'potapov_tree_height',
    'ccdc_magnitude',
    'ltr_magnitude', 'ltr_dur', 'ltr_rate', 
    'cusum_confidence', 'cusum_magnitude', 
    'ts_mean', 'ts_sd', 'ts_min', 'ts_max', 
    'bs_slope_mean', 'bs_slope_sd', 'bs_slope_min', 'bs_slope_max'
]

### 3 Vérifiez les NaN

Le processus de clustering n'accepte les NaN dans aucun des champs. Il existe 2 stratégies :

1. Supprimez toutes les lignes contenant des NaN
2. Remplacez tous les NaN par un nombre

In [ ]:
print(' Length of original dataframe: ' + str(len(df)))
df_1 = df.copy()
print(' Length of nan-removed dataframe: ' + str(len(df_1[cols_to_cluster].dropna())))

for col in cols_to_cluster:
    print(f' Column {col} contains {len(df_1[df_1[col].isna()])} NaNs')
    # print(f' Column {col} contains {len(df_1[df_1[col].isin([np.inf, -np.inf])])} Infinites')

# 2 K-Means Clustering

In [ ]:
nr_of_cluster=12

# run kmeans
kmeans = KMeans(n_clusters=nr_of_cluster, random_state=42).fit(df[cols_to_cluster])

#------------------------------------------------
# Standardize the data
#X_std = StandardScaler().fit_transform(df[cols_to_cluster])
# run kmeans with standardized data
#kmeans = KMeans(n_clusters=nr_of_cluster, random_state=42).fit(X_std)
#------------------------------------------------

# add the cluster column
df['Kmeans'] = kmeans.predict(df[cols_to_cluster])

# print number of points per clusters
clusters, counts = np.unique(df.Kmeans, return_counts=True)
print(clusters)
print(counts)

# plot data
pd.DataFrame({'counts': counts}).plot(kind='bar', title='Nr. of Points per cluster', figsize=(10,5))

# 3 parcelles

## 3.1 Statistiques et tracé de chaque cluster

In [ ]:
cols_to_plot = cols_to_cluster

# in case you want to have that different
#cols_to_plot = [
#    'mon_images',
#    'cusum_confidence', 'cusum_magnitude', 
#    'ts_mean', 'ts_sd', 'ts_min', 'ts_max', 
#    'bs_slope_mean', 'bs_slope_sd', 'bs_slope_min', 'bs_slope_max'
#]


fig, axs = h.plot_stats_per_class(df, 'Kmeans', cols_to_plot)

#### Sauver la figure

In [ ]:
from pathlib import Path

Path.cwd().joinpath('plots').mkdir(exist_ok=True)
# to save a figure of a certain column/attribute
col = 'mon_images' 

fig[col].savefig(f'plots/{col}.png')

## 3.2 Mettre en avant un cluster spécifique sur une carte

In [ ]:
cluster_to_highlight = 4

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
df.plot(ax=ax, column='Kmeans', legend=True, markersize=.1)
df[df['Kmeans']==cluster_to_highlight].plot(ax=ax, markersize=5, facecolor='red')
plt.tight_layout()

# 4 Sélectionnez un sous-ensemble d'échantillons pour chaque cluster

In [ ]:
nr_of_samples_per_cluster = 25
subset_df = pd.DataFrame(columns=df.columns)

for cluster in df.Kmeans.unique():
    
    if len(df[df.Kmeans == cluster]) < nr_of_samples_per_cluster:
        
        subset_df = pd.concat([
            subset_df,
            df[df.Kmeans == cluster].sample(len(df[df.Kmeans == cluster]))
        ])
    else:
        
        subset_df = pd.concat([
            subset_df,
            df[df.Kmeans == cluster].sample(nr_of_samples_per_cluster)
        ])
    
print(f'{len(subset_df)} samples have been selected in total')

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
subset_df = gpd.GeoDataFrame(subset_df, geometry='geometry')
subset_df.plot(column='Kmeans', ax=ax, legend=True, markersize=5)

# 5 Convertir en fichier to CEO

In [ ]:
out_csv_file = '/home/sepal-user/sbae_point_analysis_CIV/erp_5km/erp_cluster_pts.csv'

subset_df['LON'] = gpd.GeoDataFrame(subset_df).geometry.x
subset_df['LAT'] = gpd.GeoDataFrame(subset_df).geometry.y
subset_df['PLOTID'] = gpd.GeoDataFrame(subset_df).point_id

cols = subset_df.columns.tolist()
cols = [e for e in cols if e not in ('LON', 'LAT', 'PLOTID')]
new_cols = ['LON', 'LAT', 'PLOTID'] + cols
subset_df = subset_df[new_cols]
subset_df.to_csv(out_csv_file, index=False)